# EfficientDet


Este cuaderno utiliza la API de Detección de Objetos de TensorFlow 2 para entrenar el modelo 'EfficientDet d2 768x768' con un conjunto de datos personalizado y convertirlo al formato TensorFlow Lite.

 'EfficientDet d2 768x768' se refiere a una variante específica del algoritmo EfficientDet que opera en imágenes con una resolución de 768x768 píxeles.

Al trabajar a través de este Colab, se puede crear y descargar un modelo TFLite para ejecutarlo en un PC, un teléfono Android, o un dispositivo de borde como el Raspberry Pi.

### 1. Preparar el conjunto de datos personalizado

En este cuaderno es necesario de realizar una carga de un conjunto de imágenes etiquetadas en relación con las aglomeraciones de personas desde una vista aérea. Para ello se han seleccionado varios datasets con imágenes ya etiquetadas con las carcaterísticas que se busca y se han unificado en un único conjunto de datos.

Al tratarse de un dataset bastante grande la mejor opción es cargarlo desde el drive directamente. Al tratarse de un .zip para darle uso es necesario descomprimir dicho conjunto de datos.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!mkdir /content/images
!unzip -q '.../TFG_DATASET_EFFICIENTDET.zip' -d /content/images

Como el dataset que utilizamos está formado por varios subdatasets distintos, en los que se clasifica la clase persona como "0", "h", "human", "people", ..., ejecutamos un script -py que modifica los ficheros de anotacion donde se clasifique a una persona por el nombre de clase "person". De esta forma, todos los ficheros de anotaciones clasifican el objeto persona por el mismo nombre de clase.

Además se eliminan de los ficheros de anotaciones aquellas clasificaciones a otras clases que no sean personas, ya que no nos i nteresa detectar un coche, un pájaro o una biciclea. De esta forma reducimos los ficheros de anotaciones y por tanto el tiempo de entrenamiento.

In [ ]:
!python3 '.../update_xml_files.py'

'EfficientDet d2 768x768' opera en imágenes con una resolución de 768x768píxeles.

En el archivo de configuración del modelo se incluye el siguiente mensaje: keep_aspect_ratio_resizer, que redimensiona la imagen manteniendo las proporciones entre anchura y altura intactas. El problema viene a la hora de transformar el modelo entrenado en uno TFlite ya que TensorFlow no soporta keep_aspect_ratio_resizer.

Hay otros mensajes disponibles que redimensionan una imagen pero ninguno otro lo hace manteniendo las proporciones entre anchura y altura intactas. Es por ello que antes de realizar el entrenamiento ejecutamos el siguiente script que redimensiona todas las imagenes del conjunto de datos a 768x768 y modifica su fichero de anotaciones.

In [ ]:
!python3 '.../redimensionar768x768.py'

###2. Instalar dependencias de TensorFlow Object Detection

Primero, instalar la API de Detección de Objetos de TensorFlow en esta instancia de Google Colab. Esto requiere clonar el [repositorio de modelos de TensorFlow](https://github.com/tensorflow/models) y ejecutar algunos comandos de instalación.

La última versión de TensorFlow con la que se ha verificado que funciona este Colab es TF v2.8.0.

In [ ]:
# Clona desde Github el repositorio de modelos de tensorflow
!pip uninstall Cython -y # Desinstalar Cython como solución temporal para el error "No se encuentra el módulo 'object_detection'"
!git clone --depth 1 https://github.com/tensorflow/models # La opción --depth 1 indica que solo se clonará el historial de confirmaciones más reciente, lo que ahorra tiempo y espacio.

In [ ]:
# Copiar archivos de configuración en la carpeta models/research
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
# Modificar el archivo setup.py para instalar el repositorio tf-models-official dirigido a TF v2.8.0
import re
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/models/research/setup.py', 'w') as f:
    # Establecer la ruta fine_tune_checkpoint
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

In [ ]:
# Instalar la API de Detección de Objetos (NOTE: Esta celda tarda 10 mins. aprox. en ejecutarse)

# Necesidad de hacer un cambio temporal con PyYAML porque Colab no puede instalar PyYAML v5.4.1
!pip install pyyaml==5.3
!pip install /content/models/research/

# Necesidad de retroceder a TF v2.8.0 debido a un error de compatibilidad de Colab con TF v2.10 (a partir del 10/03/22)
!pip install tensorflow==2.8.0

# Instalar la version 11.0 de CUDA (para mantener compatibilidad con TF v2.8.0)
!pip install tensorflow_io==0.23.1
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
!apt-get update && sudo apt-get install cuda-toolkit-11-0
!export LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH

### 3. Preparar los datos de entrenamiento

Hay que crear un mapa de etiquetas para el detector y convertir las imágenes en un formato de archivo de datos llamado TFRecords, que son utilizados por TensorFlow para el entrenamiento. Utilizar scripts de Python para convertir automáticamente los datos al formato TFRecord. Antes de ejecutarlos, definir un mapa de etiquetas para las clases.

In [ ]:
# Crear un archivo  "labelmap.txt" con una lista de las clases que el modelo de detección de objetos detectará
%%bash
cat <<EOF >> /content/labelmap.txt
person
EOF

In [ ]:
# Descargar y ejecutar los scripts de conversión de datos: create CSV data files and TFRecord files
!python3 '/content/gdrive/MyDrive/tfg/EfficientDet/create_csv.py'
! wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/create_tfrecord.py
!python3 create_tfrecord.py --csv_input=images/train_labels.csv --labelmap=labelmap.txt --image_dir=images/train --output_path=train.tfrecord
!python3 create_tfrecord.py --csv_input=images/valid_labels.csv --labelmap=labelmap.txt --image_dir=images/valid --output_path=valid.tfrecord

Almacenaremos las ubicaciones de los archivos TFRecord y Labelmap como variables para poder hacer referencia a ellos más adelante en esta sesión de Colab.

In [ ]:
train_record_fname = '/content/train.tfrecord'
val_record_fname = '/content/valid.tfrecord'
label_map_pbtxt_fname = '/content/labelmap.pbtxt'

### 4. Configurar el entrenamiento

En esta sección, se configura el modelo y la configuración de entrenamiento. Especificar qué modelo preentrenado de TensorFlow queremos utilizar desde el Modelo Zoo de Detección de Objetos de TensorFlow 2. Cada modelo también viene con un archivo de configuración que apunta a las ubicaciones de los archivos, establece parámetros de entrenamiento (como la tasa de aprendizaje y el número total de pasos de entrenamiento) y más. Se modificará el archivo de configuración para el trabajo de entrenamiento personalizado.

La primera sección de código enumera algunos modelos disponibles en el Modelo Zoo de TF2 y define algunos nombres de archivo que se utilizarán más adelante para descargar el modelo y el archivo de configuración. Esto facilita la gestión del modelo que se utilice y agregar otros modelos a la lista más adelante.

Establece la variable "chosen_model" para que coincida con el nombre del modelo con el que se desea entrenar.

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [ ]:
# A partir de Efficientdet d3+ el entrenamiento es interrumpido por falta de memoria
model_name = 'efficientdet_d2_coco17_tpu-32'
pretrained_checkpoint = 'efficientdet_d2_coco17_tpu-32.tar.gz'
base_pipeline_file = 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config'

In [ ]:
# Crear la carpeta "mymodel" para guardar los pesos previamente entrenados y archivos de configuración del modelo
%mkdir /content/models/mymodel/
%cd /content/models/mymodel/

# Descargar los pesos del modelo previamente entrenados
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Descargar el archivo de configuración del modelo
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

Ahora que se ha descargado el modelo elegido y archivo de configuración, se necesita modificar el archivo de configuración con algunos parámetros de entrenamiento de alto nivel. Las siguientes variables se utilizan para controlar los pasos de entrenamiento:

* num_steps: La cantidad total de pasos a utilizar para entrenar el modelo. Usar más pasos si las métricas de pérdida siguen disminuyendo cuando finaliza el entrenamiento. Cuantos más pasos, más tiempo llevará el entrenamiento. El entrenamiento también puede detenerse prematuramente si la pérdida se estabiliza antes de alcanzar el número especificado de pasos.

* batch_size: El número de imágenes a utilizar por paso de entrenamiento. Un tamaño de lote más grande permite que un modelo se entrene en menos pasos, pero el tamaño está limitado por la memoria de la GPU disponible para el entrenamiento. Con las GPU utilizadas en las instancias de Colab, 16 es un buen número para modelos SSD y 4 es bueno para modelos EfficientDet por falta de memoria.









In [ ]:
num_steps = 48325
batch_size = 4

In [ ]:
# Establecer las ubicaciones de los archivos y obtener la cantidad de clases para el archivo de configuración
pipeline_fname = '/content/models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total classes:', num_classes)

Total classes: 1


In [ ]:
# (Opcional) Mostrar el contenido del archivo de configuración personalizado
import re

%cd /content/models/mymodel
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    s = re.sub(r'image_resizer\s*{\s*(.*?)\s*}',
           r'image_resizer{\nfixed_shaped_resizer{\nwidth:768\n weight:768\n}\n}', s, flags=re.DOTALL)

    s = re.sub('max_detections_per_class: [0-9]+',
               'max_detections_per_class: {}'.format(2000), s)

    s = re.sub('max_total_detections: [0-9]+',
               'max_total_detections: {}'.format(2000), s)

    s = re.sub('max_number_of_boxes: [0-9]+',
               'max_number_of_boxes: {}'.format(2000), s)

    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    s = re.sub('total_steps: [0-9]+',
               'total_steps: {}'.format(num_steps), s)

    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    f.write(s)


In [ ]:
# (Optional) Display the custom configuration file's contents
!cat /content/models/mymodel/pipeline_file.config

In [ ]:
# Establecer la ruta al archivo de configuración personalizado y el directorio para almacenar los puntos de control de entrenamiento
pipeline_file = '/content/models/mymodel/pipeline_file.config'
model_dir = '/content/training/'

## 5. Entrenar el modelo TFLite personalizado para la detección de objetos

¡Estamos listos para entrenar el modelo de detección de objetos! Antes de comenzar a entrenar, carguemos una sesión de TensorBoard para monitorear el progreso del entrenamiento.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/content/training/train'

In [ ]:
# Ejecutar entrenamiento!
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1

## 6.Convertir el modelo a TensorFlow Lite

Nuestro modelo está entrenado y listo para ser utilizado para detectar objetos. Primero, necesitamos exportar el gráfico del modelo (un archivo que contiene información sobre la arquitectura y los pesos) a un formato compatible con TensorFlow Lite. Haremos esto usando el script export_tflite_graph_tf2.py.

**Importante: antes de ejecutar la siguiente celda es necesario cambiar la config del archivo export_tflite_graph_tf2.py, en concreto el número máximo de detcciones que es capaz de realiza el modelo**

In [ ]:
# Crear un directorio para almacenar el modelo TFLite entrenado
!mkdir /content/custom_model_lite
output_directory = '/content/custom_model_lite'

# Ruta al directorio de entrenamiento
last_model_path = '/content/training'

!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

In [ ]:
# Convertir el archivo exportado en un archivo de modelo TFLite
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/content/custom_model_lite/saved_model')
tflite_model = converter.convert()

with open('/content/custom_model_lite/detect.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
!zip -r /content/training.zip /content/training
!zip -r /content/custom_model_lite.zip /content/custom_model_lite


In [ ]:
!cp  /content/custom_model_lite.zip ...
!cp  /content/training.zip ...

## 7.Calcular mAP

In [ ]:
 # Script to run custom TFLite model on test images to detect objects
# Source: https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/blob/master/TFLite_detection_image.py

# Import packages
import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

%matplotlib inline

def count_person_labels(xml_file):
  tree = ET.parse(xml_file)
  root = tree.getroot()

  person_count = 0
  for obj in root.findall('object'):
      name = obj.find('name').text
      if name == 'person':
          person_count += 1

  return person_count

### Define function for inferencing with TFLite model and displaying results

def tflite_detect_images(modelpath, folderpath, lblpath, min_conf=0.01, num_test_images=1, savepath='/content/results', txt_only=False):

  print(modelpath)
  print(folderpath)
  print(lblpath)
  print(min_conf)
  print(num_test_images)

  # Grab filenames of all images in test folder
  images = glob.glob(folderpath + '/*.jpg') + glob.glob(folderpath + '/*.JPG') + glob.glob(folderpath + '/*.png') + glob.glob(folderpath + '/*.bmp')

  # Load the label map into memory
  with open(lblpath, 'r') as f:
      labels = [line.strip() for line in f.readlines()]
  print('Longitud labels: ' + str(len(labels)))
  # Load the Tensorflow Lite model into memory
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Get model details
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  print(output_details)
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  float_input = (input_details[0]['dtype'] == np.float32)

  input_mean = 127.5
  input_std = 127.5

  # Randomly select test images
  images_to_test = random.sample(images, num_test_images)

  # Loop over every image and perform detection
  for image_path in images_to_test:
      print('Imagen: ' + image_path)
      # Obtener el nombre del archivo XML correspondiente
      xml_file = os.path.join(folderpath, os.path.splitext(image_path)[0] + '.xml')
      aforo_real = count_person_labels(xml_file)
      print(xml_file)

      # Load image and resize to expected shape [1xHxWx3]
      image = cv2.imread(image_path)
      image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      imH, imW, _ = image.shape
      image_resized = cv2.resize(image_rgb, (width, height))
      input_data = np.expand_dims(image_resized, axis=0)

      # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
      if float_input:
          input_data = (np.float32(input_data) - input_mean) / input_std

      # Perform the actual detection by running the model with the image as input
      interpreter.set_tensor(input_details[0]['index'],input_data)
      interpreter.invoke()

      # Retrieve detection results
      boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates of detected objects
      classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
      scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

      detections = []

      print(aforo_real)
      aforo = 0

      # Loop over all detections and draw detection box if confidence is above minimum threshold
      for i in range(len(scores)):
          if ((scores[i] > min_conf) and (scores[i] <= 1.0)):
              aforo += 1
              # Get bounding box coordinates and draw box
              # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
              ymin = int(max(1,(boxes[i][0] * imH)))
              xmin = int(max(1,(boxes[i][1] * imW)))
              ymax = int(min(imH,(boxes[i][2] * imH)))
              xmax = int(min(imW,(boxes[i][3] * imW)))

              cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

              # Draw label
              #object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
              label = '%s: %d%%' % ('person', int(scores[i]*100)) # Example: 'person: 72%'
              labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
              label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
              #cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
              #cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

              detections.append(['person', scores[i], xmin, ymin, xmax, ymax])

      print(aforo)

      # All the results have been drawn on the image, now display the image
      if txt_only == False: # "text_only" controls whether we want to display the image results or just save them in .txt files
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12,16))
        plt.imshow(image)
        plt.show()

      # Save detection results in .txt files (for calculating mAP)
      elif txt_only == True:

        # Get filenames and paths
        image_fn = os.path.basename(image_path)
        base_fn, ext = os.path.splitext(image_fn)
        txt_result_fn = base_fn +'.txt'
        txt_savepath = os.path.join(savepath, txt_result_fn)

        # Write results to text file
        # (Using format defined by https://github.com/Cartucho/mAP, which will make it easy to calculate mAP)
        with open(txt_savepath,'w') as f:
            for detection in detections:
                f.write('%s %.4f %d %d %d %d\n' % (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

  return

In [ ]:
%%bash
git clone https://github.com/Cartucho/mAP /content/mAP
cd /content/mAP
rm input/detection-results/*
rm input/ground-truth/*
rm input/images-optional/*
wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/calculate_map_cartucho.py

In [ ]:
!cp /content/images/test/* /content/mAP/input/images-optional # Copy images and xml files
!mv /content/mAP/input/images-optional/*.xml /content/mAP/input/ground-truth/  # Move xml files to the appropriate folder

In [ ]:
!python /content/mAP/scripts/extra/convert_gt_xml.py

In [ ]:
# Set up variables for running inference, this time to get detection results saved as .txt files
PATH_TO_IMAGES='/content/images/test'   # Path to test images folder
PATH_TO_MODEL='/content/detect.tflite'   # Path to .tflite model file
PATH_TO_LABELS='/content/labelmap.txt'   # Path to labelmap.txt file
PATH_TO_RESULTS='/content/mAP/input/detection-results' # Folder to save detection results in
min_conf_threshold=0.4   # Confidence threshold

# Use all the images in the test folder
image_list = glob.glob(PATH_TO_IMAGES + '/*.jpg') + glob.glob(PATH_TO_IMAGES + '/*.JPG') + glob.glob(PATH_TO_IMAGES + '/*.png') + glob.glob(PATH_TO_IMAGES + '/*.bmp')
images_to_test = 173 # If there are more than 500 images in the folder, just use 500

# Tell function to just save results and not display images
txt_only = True

# Run inferencing function!
print('Starting inference on %d images...' % images_to_test)
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test, PATH_TO_RESULTS, txt_only)
print('Finished inferencing!')

In [ ]:
%cd /content/mAP
!python calculate_map_cartucho.py --labels=/content/labelmap.txt